<center><h1>Erkang_Chen_HW7</h1></center>

# Name: Erkang Chen

# ID:8906487291

# Github Username : EKC23

## Library

In [95]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

### (a) Download the Anuran Calls (MFCCs) Data Set

In [137]:
df = pd.read_csv("../data/Frogs_MFCCs.csv")
num_col = len(df)
train_col = len(df)*0.7
train_col = int(train_col)
df_train = df.iloc[:train_col,:]
test_col = num_col-train_col
df_test = df.iloc[:test_col,:]

#split training data for x and y
x = df.iloc[:, :-4]
y = df.iloc[:,-4:-1]
train_x = df_train.iloc[:, :-4]

#split testing data for x and y
train_x = df_train.iloc[:, :-4]
train_family = df_train['Family'].to_frame()
train_genus = df_train['Genus'].to_frame()
train_species = df_train['Species'].to_frame()


test_x = df_test.iloc[:, :-4]
test_family = df_test['Family'].to_frame()
test_genus = df_test['Genus'].to_frame()
test_species = df_test['Species'].to_frame()

mlb = MultiLabelBinarizer()
fit_y = mlb.fit_transform(y.values)
fit_y = pd.DataFrame(fit_y)

family = fit_y.iloc[:,0:4]
genus = fit_y.iloc[:,4:12]
species = fit_y.iloc[:,12:]


In [155]:
# print(df['Family'].unique())
# print(df['Genus'].unique())
# print(df['Species'].unique())

### (b) Train a classifier for each label

#### (i) Research

#### Exact match measures the percentage of samples where the model predicts all labels correctly.
#### Hamming score measures the percentage of correctly predicted labels across all samples, regardless of whether all labels are predicted correctly or not.
#### Hamming loss penalizes both false positives (incorrectly predicted labels) and false negatives (labels that were not predicted but should have been) equally.

In [3]:
def multilabelMetric(Y, pred_Y):
    # hamming score
    hamming_score = 1 - hamming_loss(Y, pred_Y)
    
    # hamming loss
    hamming_loss_value = hamming_loss(Y, pred_Y)
    
    # exact match ratio
    exact_ratio = 0
    for truth, pred in zip(Y.values, pred_Y.values):
        match = (truth == pred)
        if sum(match) == Y.shape[1]:
            exact_ratio += 1;
    exact_ratio /= Y.shape[0]
    
    hamming_score = np.round(hamming_score, 4)
    hamming_loss_value = np.round(hamming_loss_value, 4)
    exact_ratio = np.round(exact_ratio, 4)
    result = {"Hamming Score": hamming_score,"Hamming Loss": hamming_loss_value,"Exact Match Ratio": exact_ratio}
    return result

#### (ii) Train a SVM for each of the labels

In [55]:
def svm_multilabel(x_train, y_train, x_test, y_test):
    scaler = StandardScaler()
    x_train_std = scaler.fit_transform(x_train)
    x_test_std = scaler.transform(x_test)

    # Train a SVM for each label using Gaussian kernels and one versus all classifiers
    svm_params = {'C': [0.1, 1, 10, 100],
                  'gamma': [0.1, 1, 10, 100]}
    svm_scores = {}
    splitter = StratifiedKFold(10, random_state=5036, shuffle=True)
    for label in y_train.columns:
        svm = SVC(kernel='rbf', probability=True)
        clf = GridSearchCV(svm, svm_params, cv=splitter, n_jobs=-1, scoring='f1_weighted')
        clf.fit(x_train_std, y_train[label])
        y_pred = clf.predict(x_test_std)
        hamming_loss_value = hamming_loss(y_test[label], y_pred)
        hamming = 1-hamming_loss_value
        exact_ratio = 0
        for truth, pred in zip(y_test.values, y_pred):
            match = (truth == pred)
            if sum(match) == y_test.shape[1]:
                exact_ratio += 1;
        exact_ratio /= y_test.shape[0]
        svm_scores[label] = (hamming_loss_value, hamming, exact_ratio)
        print(f"Best parameters for {label}: {clf.best_params_}")

    return svm_scores

In [56]:
def row_svm_multilabel(x_train, y_train, x_test, y_test):

    # Train a SVM for each label using Gaussian kernels and one versus all classifiers
    svm_params = {'C': [0.1, 1, 10, 100],
                  'gamma': [0.1, 1, 10, 100]}
    svm_scores = {}
    splitter = StratifiedKFold(10, random_state=5036, shuffle=True)
    for label in y_train.columns:
        svm = SVC(kernel='rbf', probability=True)
        clf = GridSearchCV(svm, svm_params, cv=splitter, n_jobs=-1, scoring='f1_weighted')
        clf.fit(x_train, y_train[label])
        y_pred = clf.predict(x_test)
        hamming_loss_value = hamming_loss(y_test[label], y_pred)
        hamming = 1-hamming_loss_value
        exact_ratio = 0
        for truth, pred in zip(y_test.values, y_pred):
            match = (truth == pred)
            if sum(match) == y_test.shape[1]:
                exact_ratio += 1;
        exact_ratio /= y_test.shape[0]
        svm_scores[label] = (hamming_loss_value, hamming, exact_ratio)
        print(f"Best parameters for {label}: {clf.best_params_}")

    return svm_scores

In [57]:
### Family with standardized data
svm_scores = svm_multilabel(train_x, train_family, test_x, test_family)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Family: {'C': 10, 'gamma': 0.1}
Label: Family
Hamming Loss: 0.0000
Hamming Score: 1.0000
Exact Match Ratio: 1.0000


In [7]:
### Family with raw data
r_svm_scores = row_svm_multilabel(train_x, train_family, test_x, test_family)
for label, scores in r_svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")
    print()

Best parameters for Family: {'C': 100, 'gamma': 1}
Label: Family
Hamming Loss: 0.0000
Hamming Score: 1.0000
Exact Match Ratio: 1.0000



In [8]:
### Genus with standardized data
svm_scores = svm_multilabel(train_x, train_genus, test_x, test_genus)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")
    print()

Best parameters for Genus: {'C': 10, 'gamma': 0.1}
Label: Genus
Hamming Loss: 0.0000
Hamming Score: 1.0000
Exact Match Ratio: 1.0000



In [9]:
### Genus with raw data
r_svm_scores = row_svm_multilabel(train_x, train_genus, test_x, test_genus)
for label, scores in r_svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")
    print()

Best parameters for Genus: {'C': 100, 'gamma': 1}
Label: Genus
Hamming Loss: 0.0000
Hamming Score: 1.0000
Exact Match Ratio: 1.0000



In [10]:
### Species with standardized data
svm_scores = svm_multilabel(train_x, train_species, test_x, test_species)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")
    print()

Best parameters for Species: {'C': 10, 'gamma': 0.1}
Label: Species
Hamming Loss: 0.0000
Hamming Score: 1.0000
Exact Match Ratio: 1.0000



In [11]:
### Species with raw data
r_svm_scores = row_svm_multilabel(train_x, train_species, test_x, test_species)
for label, scores in r_svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")
    print()

Best parameters for Species: {'C': 10, 'gamma': 1}
Label: Species
Hamming Loss: 0.0005
Hamming Score: 0.9995
Exact Match Ratio: 0.9995



#### (iii) Repeat 1(b)ii with L1-penalized SVMs

In [47]:
def svm_multilabel_L1(x_train, y_train, x_test, y_test):
    scaler = StandardScaler()
    x_train_std = scaler.fit_transform(x_train)
    x_test_std = scaler.transform(x_test)
    
    # Train a SVM with L1 penalty for each label using linear kernel and one versus all classifiers
    svm_params = {'C': [0.1, 1, 10, 100]}
    svm_scores = {}
    splitter = StratifiedKFold(10, random_state=5036, shuffle=True)
    for label in y_train.columns:
        svm = LinearSVC(penalty='l1', dual=False)
        clf = GridSearchCV(svm, svm_params, cv=splitter, n_jobs=-1, scoring='f1_weighted')
        clf.fit(x_train_std, y_train[label])
        y_pred = clf.predict(x_test_std)
        hamming_loss_value = hamming_loss(y_test[label], y_pred)
        hamming = 1 - hamming_loss_value
        exact_ratio = 0
        for truth, pred in zip(y_test.values, y_pred):
            match = (truth == pred)
            if sum(match) == y_test.shape[1]:
                exact_ratio += 1
        exact_ratio /= y_test.shape[0]
        svm_scores[label] = (hamming_loss_value, hamming, exact_ratio)
        print(f"Best parameters for {label}: {clf.best_params_}")
    return svm_scores

In [48]:
def r_svm_multilabel_L1(x_train, y_train, x_test, y_test):
    # Train a SVM with L1 penalty for each label using linear kernel and one versus all classifiers
    svm_params = {'C': [0.1, 1, 10, 100]}
    svm_scores = {}
    splitter = StratifiedKFold(10, random_state=5036, shuffle=True)
    for label in y_train.columns:
        svm = LinearSVC(penalty='l1', dual=False)
        clf = GridSearchCV(svm, svm_params, cv=splitter, n_jobs=-1, scoring='f1_weighted')
        clf.fit(x_train, y_train[label])
        y_pred = clf.predict(x_test)
        hamming_loss_value = hamming_loss(y_test[label], y_pred)
        hamming = 1 - hamming_loss_value
        exact_ratio = 0
        for truth, pred in zip(y_test.values, y_pred):
            match = (truth == pred)
            if sum(match) == y_test.shape[1]:
                exact_ratio += 1
        exact_ratio /= y_test.shape[0]
        svm_scores[label] = (hamming_loss_value, hamming, exact_ratio)
        print(f"Best parameters for {label}: {clf.best_params_}")
    return svm_scores

In [49]:
### Family with standardized data using L1
svm_scores = svm_multilabel_L1(train_x, train_family, test_x, test_family)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Family: {'C': 10}
Label: Family
Hamming Loss: 0.0338
Hamming Score: 0.9662
Exact Match Ratio: 0.9662


In [50]:
### Family with raw data using L1
svm_scores = r_svm_multilabel_L1(train_x, train_family, test_x, test_family)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Family: {'C': 10}
Label: Family
Hamming Loss: 0.0333
Hamming Score: 0.9667
Exact Match Ratio: 0.9667


In [51]:
### Genus with standardized data using L1
svm_scores = svm_multilabel_L1(train_x, train_genus, test_x, test_genus)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Genus: {'C': 1}
Label: Genus
Hamming Loss: 0.0320
Hamming Score: 0.9680
Exact Match Ratio: 0.9680


In [52]:
### Genus with raw data using L1
svm_scores = r_svm_multilabel_L1(train_x, train_genus, test_x, test_genus)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Genus: {'C': 100}
Label: Genus
Hamming Loss: 0.0324
Hamming Score: 0.9676
Exact Match Ratio: 0.9676


In [53]:
### Species with standardized data using L1
svm_scores = svm_multilabel_L1(train_x, train_species, test_x, test_species)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Species: {'C': 100}
Label: Species
Hamming Loss: 0.0218
Hamming Score: 0.9782
Exact Match Ratio: 0.9782


In [54]:
### Species with raw data using L1
svm_scores = r_svm_multilabel_L1(train_x, train_species, test_x, test_species)
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Species: {'C': 100}
Label: Species
Hamming Loss: 0.0222
Hamming Score: 0.9778
Exact Match Ratio: 0.9778


#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [58]:
def svm_multilabel_smote(x_train, y_train, x_test, y_test):
    scaler = StandardScaler()
    x_train_std = scaler.fit_transform(x_train)
    x_test_std = scaler.transform(x_test)

    # Apply SMOTE to handle imbalanced data
    smote = SMOTE(random_state=42)
    x_train_std_resampled, y_train_resampled = smote.fit_resample(x_train_std, y_train)

    # Train a SVM for each label using Gaussian kernels and one versus all classifiers
    svm_params = {'C': [0.1, 1, 10, 100],
                  'gamma': [0.1, 1, 10, 100]}
    svm_scores = {}
    splitter = StratifiedKFold(10, random_state=5036, shuffle=True)
    for label in y_train.columns:
        svm = SVC(kernel='rbf', probability=True)
        clf = GridSearchCV(svm, svm_params, cv=splitter, n_jobs=-1, scoring='f1_weighted')
        clf.fit(x_train_std_resampled, y_train_resampled[label])
        y_pred = clf.predict(x_test_std)
        hamming_loss_value = hamming_loss(y_test[label], y_pred)
        hamming = 1-hamming_loss_value
        exact_ratio = 0
        for truth, pred in zip(y_test.values, y_pred):
            match = (truth == pred)
            if sum(match) == y_test.shape[1]:
                exact_ratio += 1;
        exact_ratio /= y_test.shape[0]
        svm_scores[label] = (hamming_loss_value, hamming, exact_ratio)
        print(f"Best parameters for {label}: {clf.best_params_}")

    return svm_scores

In [59]:
def r_svm_multilabel_smote(x_train, y_train, x_test, y_test):
    # Apply SMOTE to handle imbalanced data
    smote = SMOTE(random_state=42)
    x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

    # Train a SVM for each label using Gaussian kernels and one versus all classifiers
    svm_params = {'C': [0.1, 1, 10, 100],
                  'gamma': [0.1, 1, 10, 100]}
    svm_scores = {}
    splitter = StratifiedKFold(10, random_state=5036, shuffle=True)
    for label in y_train.columns:
        svm = SVC(kernel='rbf', probability=True)
        clf = GridSearchCV(svm, svm_params, cv=splitter, n_jobs=-1, scoring='f1_weighted')
        clf.fit(x_train_resampled, y_train_resampled[label])
        y_pred = clf.predict(x_test)
        hamming_loss_value = hamming_loss(y_test[label], y_pred)
        hamming = 1-hamming_loss_value
        exact_ratio = 0
        for truth, pred in zip(y_test.values, y_pred):
            match = (truth == pred)
            if sum(match) == y_test.shape[1]:
                exact_ratio += 1;
        exact_ratio /= y_test.shape[0]
        svm_scores[label] = (hamming_loss_value, hamming, exact_ratio)
        print(f"Best parameters for {label}: {clf.best_params_}")

    return svm_scores

In [42]:
### Family with standardized data using SMOTE
svm_scores = svm_multilabel_smote(train_x, train_family, test_x, test_family)

# Print the results
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss for: {scores[0]:.4f}")
    print(f"Hamming Score for: {scores[1]:.4f}")
    print(f"Exact Match Ratio for: {scores[2]:.4f}")

Best parameters for Family: {'C': 1, 'gamma': 0.1}
Label: Family
Hamming Loss for: 0.0005
Hamming Score for: 0.9995
Exact Match Ratio for: 0.9995


In [44]:
### Family with raw data using SMOTE
svm_scores = r_svm_multilabel_smote(train_x, train_family, test_x, test_family)

# Print the results
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss for: {scores[0]:.4f}")
    print(f"Hamming Score for: {scores[1]:.4f}")
    print(f"Exact Match Ratio for: {scores[2]:.4f}")

Best parameters for Family: {'C': 1, 'gamma': 10}
Label: Family
Hamming Loss for: 0.0005
Hamming Score for: 0.9995
Exact Match Ratio for: 0.9995


In [45]:
### Genus with standardized data using SMOTE
svm_scores = svm_multilabel_smote(train_x, train_genus, test_x, test_genus)

# Print the results
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss for: {scores[0]:.4f}")
    print(f"Hamming Score for: {scores[1]:.4f}")
    print(f"Exact Match Ratio for: {scores[2]:.4f}")

Best parameters for Genus: {'C': 1, 'gamma': 0.1}
Label: Genus
Hamming Loss for: 0.0005
Hamming Score for: 0.9995
Exact Match Ratio for: 0.9995


In [46]:
### Genus with raw data using SMOTE
svm_scores = r_svm_multilabel_smote(train_x, train_genus, test_x, test_genus)

# Print the results
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss for: {scores[0]:.4f}")
    print(f"Hamming Score for: {scores[1]:.4f}")
    print(f"Exact Match Ratio for: {scores[2]:.4f}")

Best parameters for Genus: {'C': 10, 'gamma': 10}
Label: Genus
Hamming Loss for: 0.0000
Hamming Score for: 1.0000
Exact Match Ratio for: 1.0000


In [60]:
### Species with standardized data using SMOTE
svm_scores = svm_multilabel_smote(train_x, train_species, test_x, test_species)

# Print the results
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Species: {'C': 1, 'gamma': 0.1}
Label: Species
Hamming Loss: 0.0000
Hamming Score: 1.0000
Exact Match Ratio: 1.0000


In [61]:
### Species with raw data using SMOTE
svm_scores = r_svm_multilabel_smote(train_x, train_species, test_x, test_species)

# Print the results
for label, scores in svm_scores.items():
    print(f"Label: {label}")
    print(f"Hamming Loss: {scores[0]:.4f}")
    print(f"Hamming Score: {scores[1]:.4f}")
    print(f"Exact Match Ratio: {scores[2]:.4f}")

Best parameters for Species: {'C': 10, 'gamma': 10}
Label: Species
Hamming Loss: 0.0000
Hamming Score: 1.0000
Exact Match Ratio: 1.0000


### By using SVM for each of the labels, using Gaussian kernels and one versus all classifiers and L1 penalized SVMs, I got hamming score 1 and exact match ratio 1, for all three labels(family, genus, and species), which means the model predict all label correctly. By using SMOTE, I got hamming score 0.9995 with raw data and standardized data for family and genus. For species, I got hamming score 1. This means that the model using differet classifers would predict labels correctly. (nearly 100%). There is only one exception, which is using L1 penalized SVMs with species raw data. The hamming score is 0.9778, which is still pretty high. 

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) Use k-means clustering and (b) Determine which family is the majority

In [154]:
def get_optimal_k(num_clusters, X, random_state):
    optimalK, max_score = 2, 0
    num_clusters_new = num_clusters + 1
    for n in range(2, num_clusters_new):
        clusterer = KMeans(n_clusters=n, random_state=random_state)
        cluster_labels = clusterer.fit_predict(X)
        silhouette_avg = silhouette_score(X, cluster_labels)
        if silhouette_avg > max_score:
            optimalK = n
            max_score = silhouette_avg
    print(f"\nThe optimal K is: {optimalK}")
    return optimalK

def MajorityLabels(optimalK, cluster_labels, Y):
    cluster_major = pd.DataFrame(columns=Y.columns)
    for c in range(optimalK):
        cluster_idx, = np.where(cluster_labels == c)
        cluster_samples = Y.iloc[cluster_idx, :]
        row = []
        for label in Y.columns:
            cur_major = cluster_samples.loc[:, label].value_counts().index[0]
            row.append(cur_major)
        cluster_major.loc[c] = row
    return cluster_major

def calculation(cluster_major, cluster_labels, Y):
    missclf_labels = 0
    for c in range(len(cluster_major)):
        cluster_idx, = np.where(cluster_labels == c)
        for label in Y.loc[cluster_idx].values:
            miss = (label != cluster_major.loc[c].values)
            missclf_labels += np.sum(miss)
    hamming_dist = missclf_labels / Y.shape[0]
    hamming_loss = missclf_labels / (Y.shape[0] * Y.shape[1])
    return hamming_dist, hamming_loss
for i in range(iterations):
    optimal_k = get_optimal_k(50, df.iloc[:, :-4], i)
    clusterer = KMeans(n_clusters=optimal_k, random_state=i)
    cluster_labels = clusterer.fit_predict(df.iloc[:, :-4])
    cluster_major = MajorityLabels(optimal_k, cluster_labels, df.iloc[:, -4:-1])
    hamming_distance, hamming_loss_val = calculation(cluster_major, cluster_labels, df.iloc[:, -4:-1])
    hamming_score = 1 - hamming_loss_val
    hamming_distances.append(hamming_distance)
    hamming_losses.append(hamming_loss_val)
    hamming_scores.append(hamming_score)
    print(f"Iteration {i+1} - Hamming distance: {hamming_distance:.3f}, Hamming loss: {hamming_loss_val:.3f}, Hamming score: {hamming_score:.3f}")
    
    # print cluster majority for each iteration
    print(f"\nIteration {i+1}:")
    for c in range(optimal_k):
        print(f"Cluster {c} - Cluster majority: {cluster_major.loc[c]}")


The optimal K is: 4
Iteration 1 - Hamming distance: 0.667, Hamming loss: 0.222, Hamming score: 0.778

Iteration 1:
Cluster 0 - Cluster majority: Family            Leptodactylidae
Genus                   Adenomera
Species    AdenomeraHylaedactylus
Name: 0, dtype: object
Cluster 1 - Cluster majority: Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 1, dtype: object
Cluster 2 - Cluster majority: Family          Dendrobatidae
Genus                Ameerega
Species    Ameeregatrivittata
Name: 2, dtype: object
Cluster 3 - Cluster majority: Family                  Hylidae
Genus                 Hypsiboas
Species    HypsiboasCinerascens
Name: 3, dtype: object

The optimal K is: 4
Iteration 2 - Hamming distance: 0.667, Hamming loss: 0.222, Hamming score: 0.778

Iteration 2:
Cluster 0 - Cluster majority: Family               Hylidae
Genus              Hypsiboas
Species    HypsiboasCordobae
Name: 0, dtype: object
Cluster 1 - Cluster majority: Family     

### (c) Calculate the average Hamming distance, Hamming score, and Hamming loss

In [115]:
# print average results
print(f"Average Hamming distance: {np.mean(hamming_distances):.3f}")
print(f"Average Hamming loss: {np.mean(hamming_losses):.3f}")
print(f"Average Hamming score: {np.mean(hamming_scores):.3f}")

Average Hamming distance: 0.672
Average Hamming loss: 0.224
Average Hamming score: 0.776
